<a href="https://colab.research.google.com/github/SoumikSarkar830/Laser_PINN/blob/main/LaserPINN_Zigzag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
class PINN(nn.Module):
    def __init__(self):
      super(PINN, self).__init__()
      self.net = nn.Sequential(
          nn.Linear(3, 100),  # x,y,t as inputs
          nn.Tanh(),
          nn.Linear(100, 100),
          nn.Tanh(),
          nn.Linear(100, 100),
          nn.Tanh(),
          nn.Linear(100, 100),
          nn.Tanh(),
          nn.Linear(100, 1)  # Temparaure T as output
      )

    def forward(self, x):
      return self.net(x)


In [3]:
model = PINN()
model.to(device)

PINN(
  (net): Sequential(
    (0): Linear(in_features=3, out_features=100, bias=True)
    (1): Tanh()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): Tanh()
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): Tanh()
    (6): Linear(in_features=100, out_features=100, bias=True)
    (7): Tanh()
    (8): Linear(in_features=100, out_features=1, bias=True)
  )
)

In [4]:
stefanBoltz = 5.670373e-8
hCoeff = 1
emiss = .5
ta = 300

In [21]:
def heat_loss(model, x, y, t, alpha = 0.01):

    # PDE Loss

    inputs = (torch.cat((x, y, t), dim=1))
    T = model(inputs)

    dT_dx = torch.autograd.grad(T, x, torch.ones_like(T), create_graph=True)[0]
    dT_dy = torch.autograd.grad(T, y, torch.ones_like(T), create_graph=True)[0]
    dT_dt = torch.autograd.grad(T, t, torch.ones_like(T), create_graph=True)[0]

    dT_dx2 = torch.autograd.grad(dT_dx, x, torch.ones_like(dT_dx), create_graph=True)[0]
    dT_dy2 = torch.autograd.grad(dT_dy, y, torch.ones_like(dT_dy), create_graph=True)[0]

    laser_heat = 2 * (emiss * stefanBoltz * (T**4 - ta**4) + hCoeff * (T - ta))

    residual = dT_dt - alpha * (dT_dx2 + dT_dy2) - laser_heat
    pde_loss = torch.mean(residual**2)


    # Dirichlet Boundary Conditions

    fixed_T = 100.0

    tol = 1e-6
    zero = torch.tensor(0.0, device=x.device)
    one = torch.tensor(1.0, device=x.device)

    bc_x = torch.isclose(x, zero, atol=tol) | torch.isclose(x, one, atol=tol)
    bc_y = torch.isclose(y, zero, atol=tol) | torch.isclose(y, one, atol=tol)
    bc = bc_x | bc_y

    bc_loss = torch.mean((model(torch.cat([x[bc].reshape(-1, 1), y[bc].reshape(-1, 1), t[bc].reshape(-1, 1)], dim=1)) - fixed_T)**2)

    # Neumann Boundary Conditions

    neumann_loss_x = torch.mean(dT_dx[bc_x]**2)
    neumann_loss_y = torch.mean(dT_dy[bc_y]**2)
    neumann_loss = neumann_loss_x + neumann_loss_y





    return pde_loss + bc_loss + neumann_loss









In [13]:
grid_size = 100
time_steps = 100

n_rows = grid_size // 5

x_data, y_data, t_data = [],[],[]
laser_path = []

for i in range(0,n_rows,2):
    y1 = i / n_rows -1
    # Left to right
    for x in np.linspace(0, 1, grid_size):
        laser_path.append((x, y1))
    # Right to left for the next hatch if there is one
    if i + 1 < n_rows:
        y2 = (i+1) / n_rows -1
        for x in np.linspace(1, 0, grid_size):
            laser_path.append((x, y2))

for t in range(time_steps):
    for pos in laser_path:
        x_data.append(pos[0])
        y_data.append(pos[1])
        t_data.append(t / time_steps-1)  # Normalize time

x_data = torch.tensor(x_data, dtype=torch.float32).view(-1, 1).requires_grad_(True).to(device)
y_data = torch.tensor(y_data, dtype=torch.float32).view(-1, 1).requires_grad_(True).to(device)
t_data = torch.tensor(t_data, dtype=torch.float32).view(-1, 1).requires_grad_(True).to(device)





tensor(False, device='cuda:0') tensor(False, device='cuda:0') tensor(False, device='cuda:0')
tensor(False, device='cuda:0') tensor(False, device='cuda:0') tensor(False, device='cuda:0')


In [22]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [25]:
epochs = 2000

for epoch in range(epochs):
    optimizer.zero_grad()
    loss = heat_loss(model, x_data, y_data, t_data)
    loss.backward()
    optimizer.step()



    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Testing the trained model

x_test = torch.linspace(0, 1, 100).to(device)
y_test = torch.linspace(0, 1, 100).to(device)
t_test = torch.linspace(0, 1, 20).to(device)

X_test, Y_test, T_test = torch.meshgrid(x_test, y_test, t_test, indexing="ij")

test_input = torch.cat([
    X_test.reshape(-1, 1),
    Y_test.reshape(-1, 1),
    T_test.reshape(-1, 1)
], dim=1)

with torch.no_grad():
    T_test_pred = model(test_input)
    T = torch.nan_to_num(T, nan=0.0, posinf=1.0, neginf=-1.0)

T_test_pred = T_test_pred.view(100, 100, 20).cpu()



plt.figure(figsize=(8, 8))

plt.imshow(T_test_pred[:, :, 10], cmap='viridis', origin='lower', extent=[0, 1, 0, 1])

plt.colorbar(label='Temperature')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Temperature Distribution at t = 0.5')
plt.show()

Epoch 0, Loss: nan
Epoch 100, Loss: nan
Epoch 200, Loss: nan
Epoch 300, Loss: nan
Epoch 400, Loss: nan
Epoch 500, Loss: nan
Epoch 600, Loss: nan
Epoch 700, Loss: nan
Epoch 800, Loss: nan
Epoch 900, Loss: nan
Epoch 1000, Loss: nan
Epoch 1100, Loss: nan
Epoch 1200, Loss: nan
Epoch 1300, Loss: nan
Epoch 1400, Loss: nan
Epoch 1500, Loss: nan
Epoch 1600, Loss: nan
Epoch 1700, Loss: nan
Epoch 1800, Loss: nan
Epoch 1900, Loss: nan


NameError: name 'T' is not defined